In [21]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

In [22]:
train_features_data=pd.read_csv('train_features.csv')
train_labels_data=pd.read_csv('train_labels.csv')
test_features_data=pd.read_csv('test_features.csv')
# add labels with matching pid to features
labels_copied = pd.DataFrame()
labels_copied = train_labels_data.loc[train_labels_data.index.repeat(12)]
labels_copied = labels_copied.drop(columns=['pid'])
labels = train_labels_data.drop(columns=['pid'])
LABELS1 = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
LABELS2 = ['LABEL_Sepsis']
LABELS3 = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

## Subtask 1

In [38]:
X = train_features_data.drop(columns=['pid', 'Time']).reset_index(drop=True).fillna(test_features_data.mean())
y = labels_copied[LABELS1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle=True)

TEST_X = test_features_data.drop(columns=['pid', 'Time']).reset_index(drop=True).fillna(test_features_data.mean())

### RandomForest model and Roc_Auc_Score using every hour per patient as sample for training

In [39]:
clf=RandomForestClassifier(n_estimators=300, class_weight=None, n_jobs=-1 )
clf.fit(X_train, y_train)
y_pred_proba=clf.predict_proba(X_test)
err=np.empty(10)
list_proba=list()
for i,label in enumerate(LABELS1):
    err[i]=roc_auc_score(y_test[label], y_pred_proba[i][:,1])
    list_proba.append(y_pred_proba[i][:,1])
display(err)
display('Average score: ', np.mean(err))

array([0.84238713, 0.75417446, 0.71588294, 0.71788092, 0.71524152,
       0.7659438 , 0.81023819, 0.78541174, 0.78157756, 0.90859597])

'Average score: '

0.779733425206673

### Reduce values by 12 for test-labels and y_pred_proba

In [25]:
err=np.empty(10)
for i, labels in enumerate(LABELS1):
    y_test_reduced=y_test[labels][0:len(y_test[labels]):12]
    y_pred_proba_reduced=np.empty(int(len(y_pred_proba[i][:,1])/12))
    counter=0
    for splits in np.split(np.array(y_pred_proba[i][:,1]), int(len(y_pred_proba[i][:,1])/12)):
        y_pred_proba_reduced[counter]=splits.mean() 
        counter=counter+1
    err[i]=roc_auc_score(y_test_reduced, y_pred_proba_reduced)
    display(err[i])
display('Average Score: ', np.mean(err))

0.632890365448505

0.5982732351447436

0.5375155666251556

0.5236486486486487

0.5408954279386173

...

0.575

0.37945945945945947

0.6469155844155844

'Average Score: '

0.5691216427727837

### Predictions for Test_features

In [26]:
TEST_y_pred_proba=clf.predict_proba(TEST_X)

### Reduce values by 12 for TEST_y_pred_proba

In [27]:
TEST_list_proba=list()
for j in range(10):
    TEST_y_pred_proba_reduced=np.empty(int(len(TEST_y_pred_proba[j][:,1])/12))
    counter=0
    for splits in np.split(np.array(TEST_y_pred_proba[j][:,1]), int(len(TEST_y_pred_proba[j][:,1])/12)):
        TEST_y_pred_proba_reduced[counter]=splits.mean() 
        counter=counter+1
    TEST_list_proba.append(TEST_y_pred_proba_reduced)
proba_subtask1=TEST_list_proba

## Subtask 2

In [28]:
y = labels_copied[LABELS2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle=True)

### RandomForest model and Roc_Auc_Score using every hour per patient as sample

In [29]:
clf=RandomForestClassifier(n_estimators=300, class_weight=None, n_jobs=-1)

clf.fit(X_train, y_train)
y_pred_proba=clf.predict_proba(X_test)
err=roc_auc_score(y_test, y_pred_proba[:,1])
display(err)

C:\Users\Manue\anaconda3\envs\ML\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.7120814987481655

### Prediction for Test features

In [30]:
TEST_y_pred_proba=clf.predict_proba(TEST_X)

### Reduce values by 12 for TEST_y_pred_proba

In [31]:
TEST_y_pred_proba_reduced=np.empty(int(len(TEST_y_pred_proba[:,1])/12))
counter=0
for splits in np.split(np.array(TEST_y_pred_proba[:,1]), int(len(TEST_y_pred_proba[:,1])/12)):
    TEST_y_pred_proba_reduced[counter]=splits.mean() 
    counter=counter+1
proba_subtask2=TEST_y_pred_proba_reduced

## Subtask 3

In [32]:
y = labels_copied[LABELS3]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle=True)

### Implement MultiTaskLassoCV model

In [33]:
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error


cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

model = RidgeCV(alphas=(0.1, 1, 10), cv=cv)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print(mean_squared_error(y_test, y_pred))

58.78080988076304


### Reduce values by 12 for test-labels and y_pred

In [34]:
for i, labels in enumerate(LABELS3):
    y_test_reduced=y_test[labels][0:len(y_test[labels]):12]
    y_pred_reduced=np.empty(int(len(y_pred[:,i])/12))
    counter=0
    for splits in np.split(np.array(y_pred[:,i]), int(len(y_pred[:,i])/12)):
        y_pred_reduced[counter]=splits.mean() 
        counter=counter+1
    display(mean_squared_error(y_test_reduced, y_pred_reduced))

10.31047510321523

151.526961808906

7.808054191742678

268.7465567434238

### Predictions for Test features

In [35]:
TEST_y_pred_values=model.predict(TEST_X)
display(TEST_y_pred_values)

array([[18.42989954, 84.66569541, 97.16850264, 85.20450256],
       [16.52744139, 80.99853638, 98.08966975, 80.68784629],
       [16.36000022, 83.1671906 , 98.04754418, 83.01852774],
       ...,
       [18.14832684, 77.00301936, 97.73806462, 87.6635914 ],
       [18.44623374, 79.02377686, 97.74498834, 87.84807596],
       [17.45422415, 77.13011665, 97.75621869, 86.55120625]])

### Reduce values by 12 for TEST_y_pred_values

In [36]:
TEST_list_values=list()
for j in range(4):
    TEST_y_pred_values_reduced=np.empty(int(len(TEST_y_pred_values[:,j])/12))
    counter=0
    for splits in np.split(np.array(TEST_y_pred_values[:,j]), int(len(TEST_y_pred_values[:,j])/12)):
        TEST_y_pred_values_reduced[counter]=splits.mean() 
        counter=counter+1
    TEST_list_values.append(TEST_y_pred_values_reduced)
proba_subtask3=TEST_list_values

### Write values into sample dataframe

In [37]:
filename = 'sample.zip'
df_submission = pd.read_csv(filename)
for i,label in enumerate(LABELS1):
    # round classification labels
    df_submission[label]=proba_subtask1[i]
df_submission[LABELS2[0]]=proba_subtask2
for i,label in enumerate(LABELS3):
    # round classification labels
    df_submission[label]=proba_subtask3[i]
df_submission.to_csv('submission.csv',index=False)
df_submission.to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')